<a href="https://colab.research.google.com/github/AjeetSingh02/Courses/blob/master/Intro_to_tensorflow/week2IntroToTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimator API

Estimators are a part of the high level TensorFlow APIs. Your first contact with TensorFlow in the previous module was at the Core TensorFlow level. But you essentially use TensorFlow as a numeric processing library. Below this level sits a series of APIs that you do not typically interact with. 

They handle the hardware, CPUs, GPUs, TPUs, or alternative platforms like Android. Above Core TensorFlow, you will find APIs for all the typical bricks needed to build a model. 

Different kinds of neural network layers, different loss functions, and so on. And finally, to wrap it all up, Estimators. 

**Benifits of Estimator**

As data and training time grows, however, your needs will increase. Do you need checkpoints to pause and resume your training? Estimators have them. 

Your data no longer fits in memory? Estimators are designed with a data set API that handles out of memory data sets. You can not train a large network without seeing how its doing. 

Estimators automatically surface key metrics during training that you can visualize in Tensor board. 

Are you thinking now about distributed training? Estimators come with the necessary cluster execution code already built in. 

And finally, you will want to wrap your model to make it ready for ML-Engine's hyper-parameter tuning, and maybe also push it to production behind ML-Engine's managed and autoscaled prediction service. The Estimator API has you covered there as well. 


The base class estimator lets you wrap your own model that you would build from layers using the TF layers API. 

But if you're building something fairly standard, no need to go there. TensorFlow has a set of pre-made estimators that you can try out. 

Linear or dense neural network classifiers to classify data into categories, and similar regressors to predict continuous values. 

And don't also forget the DNNLinearCombinedClassifier, also known as the wide and deep model according to the Google research paper that popularized it. This one is not trivial, we use it for example to power the recommendation engine in Google Play. But it is very flexible and has times been described as the work horse of Enterprise Machine Learning. It works for all kinds of structure data and you can use it out of the box. 

**The one thing to remember is that thanks to common API, pre-made estimators are interchangeable. It is easy to try and test them all.**

**Pre-made Estimators**

How to use them

In [0]:
import tensorflow as tf

In [0]:
# This is how we define the features
featcols = [
            tf.feature_column.numeric_column("sq_footage"),
            tf.feature_column.categorical_column_with_vocabulary_list("type", ["house", "apt"])
]

In [0]:
model = tf.estimator.LinearRegressor(featcols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpaijdo1ok', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f53f1318518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# This is how we provide input for training
# This function will return a dictionary of features with same names
# as described above and a list of corresponding labels
def train_input_fn():
    features = {
        "sq_footage": [1000, 2000, 3000, 1000, 2000, 3000],
        "type":       ["house", "house", "house", "apt", "apt", "apt"]
    }
    labels = [500, 1000, 1500, 700, 1300, 1900]

    return features, labels

In [0]:
model.train(train_input_fn, steps=100)

In [0]:
def predict_input_fn():
    features = {
        "sq_footage": [1500, 1800],
        "type": ["house", "apt"]
    }

    return features

In [0]:
predictions = model.predict(predict_input_fn)

In [0]:
print(next(predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpaijdo1ok/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([855.93005], dtype=float32)}


In [0]:
import itertools

In [0]:
# This is how you read from predictions generator

for pred in list(itertools.islice(predictions, 5)):
    print(pred)
    print(pred["predictions"])

if we run for loop on this predcitions(generator object) then it will run indefinitely giving the same values in loop

**Training on in-memory data**

Using Pandas or Numpy to feed in data

In [0]:
def pandas_train_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x = df, # Columns in dataframe with same name 
                # will be selected automatically
        y = df["price"],
        batch_size = 128,
        num_epochs = 10,
        shuffle=True,
        queue_capacity=1000
    )

In [0]:
# Variations of training
model.train(pandas_train_input_fn(df))

# 1000 additional steps from last checkpoint
model.train(pandas_train_input_fn(df), steps=1000)

# 1000 - steps done in till last checkpoint
model.train(pandas_train_input_fn(df), max_steps=1000)

**Transcript**

Let's move to real data and not the constants we have been using previously to illustrate the Estimator API. 

If your data fits a memory in the form of either numpy arrays or Pandas, the Estimator API has easy convenience functions for feeding them into your model. They are called **estimator.inputs.numpy_input_fn** and **estimator.inputs.Pandas_input_fn**. 

You specify your feature dictionary as **x** using the same names as the ones you used to, to define features columns and your labels as **y**. 

When the reading from Pandas, these feature names are not even necessary. The API will pick up the correct columns from your Pandas data frame based on the names you used when defining feature columns. These input functions implement the key functionalities needed when training neural network models. 

Typically, training works best when one training step is performed on what is called a mini batch of input data at a time, not a single data item and not the entire data set either. You can specify the batch size here. You can also say how many times you want to repeat the data set during training called the number of epochs. And finally, thoroughness shuffling your training data is important. You can ask the Estimator API to do it. And to avoid replicating the data set in memory, when doing so, you can specify the size of the shuffle queue. 

By default, training will run until your training data is exhausted or exhausted n times if you specifies you want n epochs in your input function. You can also override this with an explicit number of steps when calling the train function. There are two variants for the step setting.

Steps equals 1,000 will run 1,000 additional training steps starting from the last checkpoint. One step here corresponds to one batch of input data. Max steps equals 1,000 on the other hand restarts from the latest check points reads the step count reached in the previous run and continues until the step count reaches max steps. This can potentially do nothing if the checkpoint was already there. Here is the entire code with feature columns, a model, an input function that reads data from a Pandas data frame and the training itself. Would you like to add a new feature? Let's say the number of bedrooms in the house. You can add it to the list of feature columns and make sure it is present with the same name in the Pandas data frame.

In [0]:
# We can also read the already trained model from saved directory and resume training or
# Make predictions with that

# DataSet API

**Read one CSV file using TextLineDataset**

In [0]:
def decode_line(row):
    cols =  tf.decode_csv(row, record_defaults=[[0],['house'],[0]])
    features = {'sq_footage':cols[0], 'type':cols[1]}
    label = cols[2]
    return features, label

dataset = tf.data.TextLineDataset("train1.csv").map(decode_line)

dataset = dataset.shuffle(1000).repeat(15).batch(128)

def input_fn():
    features, label = dataset.make_one_shot_iterator().get_next()
    return features, label

model.train(input_fn)

**Read a set of sharded CSV files using TextLineDataset**

In [0]:
def decode_line(row):
    cols =  tf.decode_csv(row, record_defaults=[[0],['house'],[0]])
    features = {'sq_footage':cols[0], 'type':cols[1]}
    label = cols[2]
    return features, label

dataset = tf.data.Dataset.list_files("train.csv-*").flat_map(tf.data.TextLineDataset).map(decode_line)

dataset = dataset.shuffle(1000).repeat(15).batch(128)

def input_fn():
    features, label = dataset.make_one_shot_iterator().get_next()
    return features, label

model.train(input_fn)

**Transcripts:**

Here, we introduce the data set API, which comes with TensorFlow and is designed to be used with estimators. Let's see why and when you'll need to use it. We now know how to pick a model and how to feed it data from memory, for training and predictions. But for real world models, we still need to solve a couple of practical issues. The estimator API can help there too. Let's start with the first one. Data size, yes, in the real life, your training data will readily fit in memory and you will load it progressively from disk during training. How can you write an input function for your estimator, that does that by using data sets, as in tf.data.Dataset. Large data sets tend to be sharded into multiple files, which can be loaded progressively. Remember, that we train on mini batches of data. We do not need to have the entire data set in memory. One mini batch is all we need for one training step. Here is the data set API. It will help us create input functions for our model that load data progressively. There are a specialized data set classes, that can read data from text files like CSVs, TensorFlow records, or fixed length record files. For anything else, you can use the generic data set clause and add your own decoding code. Here is an example where we use text line data set to load data from a CSV file. Let's go through the different parts. This part of the code tells the data set class how to organize the data into training batches of 128, repeated for 15 epoch, and of course, shuffled with a shuffle buffer of 1000 elements. Here, we instantiate the text line data set from a filename. This loads the file and splits it into lines. The resulting data set is a set of text lines. We can now use the map function to transform the lines. In this case, we want to split each line into data items. Map, applies a function to each item in the data set independently. And in this function we use the TF decode CSV function to extract the comma separated values from the text lines, and format them into features and labels as our model expects them. After the map, we have a data set of featured and label base. Finally, we create the input function for our model. This piece of boilerplate code does the trick. But you might be wondering why is it called an iterator, and why get next, and what is actually going on when we train? Let's dive under the hood again. It will help us understand. TensorFlow, works with a deferred execution principle. Older tf.something commands, that you write in Python do not actually process data. They build a graph of operations in memory. This graph will be executed when we train or predict. When we instantiate an estimator, like linear or regresor the same thing happens. A TensorFlow graph is created in memory representing our model. Now, the problem is to connect it to a data source. That is what input functions are for. The contract for an input function, is to return one TensorFlow node, representing the features and labels expected by the model. This node will be connected to the inputs of the model, and its responsibility is to deliver a fresh batch of data every time it is executed, during training or inference. That is what the data set API is useful for. It generates for you input nodes that deliver one batch of data at each training step. And they also make sure the data is loaded progressively and never saturates the memory. When you call dataset.makeiterator.getnext, you're not really getting the next element in the data set, you are getting a TensorFlow node, that each time it gets executed during training returns a batch of training data. Let's recap. Input functions are called when a model is instantiated. They return a pair of TensorFlow nodes to be attached to the inputs of your model and these nodes are responsible for pumping data into your model during training or inference. There are a couple of misconceptions about input functions, that I would like to clear. Now, an input function is not called every time your model needs data. It is called only once, at model creation time. And no, input functions are not expected to return actual data, even if that is what it looks like when you write them. They return TensorFlow nodes, and these nodes return data when they get executed. You can actually place arbitrarily complex code in the input function to transform your data, as long as you bear in mind that it will run only once. Whatever transformations you want to apply, and whether you use the data set API or not, make sure they are expressed in tf.something commands that generate a TensorFlow graph. That is how you get your transformations to be applied to each batch of data, as it is loaded into your model. Even if the input function itself is only called code once. Here's the full code again. Let us go through it one more time. Starting at the bottom and going up. Muddle the train launches the training loop. The model receives data from its input nodes, features and labels as defined in the input function. These nodes iterate on the data set and return one batch of data every time that they get executed in the training loop. That explains why the name of the data set API you call to give them is data set, make one short iterator, get next. The data set shuffles the data, repeats it for 15 epochs, and batches into mini batches of 128 elements. The data set has been produced by reading lines from a text file and decoding the comma separated values from them. The map operation transforms a data set of text lines into a data set of features and labels. Finally, we have to address our initial concerns, loading large data sets from a set of sharded files. One extra line of code will do. We first scan the disk and load a data set of filenames using the data set that list files functions. It supports a globe like syntax which stars to match filenames with a common pattern. Then, we use text line data set to load these files and turn each filename into a data set of text lines. We flat map all of them together into a single data set. And then for each line of text, we use map to apply the CSV parsing algorithm and obtain a data set of features and labels. Why two mapping functions, map and flat map? Well, one of them is simply for one to one transformations and the other one for one to many transformations. Parsing a line of text int- is one to one transformation, so we apply it with map. When loading a file with text line data set, one filename becomes a collection of text lines. So, that's a one to many transformations and it is applied with flat map to flatten all the resulting text lines into one data set. Now you know how to use data sets to generate proper input functions for your models and get them training on large, out of memory data sets. But data sets also offer a rich API for working on and transforming your data. Take advantage of it.